
download dataset online:
Dataset 1: OPPORTUNITY dataset
https://archive.ics.uci.edu/ml/datasets/OPPORTUNITY +Activity+Recognition



dependency:
!pip install wget

In [1]:
#import urllib
import json
import wget
import os
import pandas as pd
import re
import numpy as np
import torch
from torch import nn
import torch.optim as optim
import torch.nn.functional as F
from random import random
import zipfile
from sklearn.metrics import f1_score

In [3]:
cfg_update = 1
if cfg_update == 1:
    cfg = {}
    cfg['OPPORTUNITY_DATASET'] = {}
    cfg['OPPORTUNITY_DATASET']['Download_URL'] = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00226/OpportunityUCIDataset.zip'
    cfg['OPPORTUNITY_DATASET']['Download_Destination'] = 'Dataset/OpportunityUCIDataset'
    cfg['OPPORTUNITY_DATASET']['dataset_folder'] = 'Dataset/OpportunityUCIDataset/OpportunityUCIDataset/dataset'
    cfg['OPPORTUNITY_DATASET']['Normalization_parameter'] = 'Dataset/OPPORTUNITY_norm_parameter.csv'
    
    cfg['SKODA_DATASET'] = {}
    cfg['SKODA_DATASET']['Download_URL'] = 'http://har-dataset.org/lib/exe/fetch.php?media=wiki:dataset:skodaminicp:skodaminicp_2015_08.zip'
    cfg['SKODA_DATASET']['Download_Destination'] = 'Dataset/SKODA_DATASET'
    
    
    
    
    
    
    
    #data = json.dumps(data, indent=4)
    #print(data)
    with open('cfg.json', 'w') as outfile:
        json.dump(cfg, outfile)
        
        
        
        
        

In [4]:
def import_cfg_file(cfg_file_path):
    with open(cfg_file_path) as json_file:
        cfg = json.load(json_file)
    return cfg

def download_dataset(Source_URL, Destination):
    if not os.path.isfile(Destination + '.zip'):
        wget.download(Source_URL, Destination + '.zip')
    with zipfile.ZipFile(Destination + '.zip', 'r') as zip_ref:
        zip_ref.extractall(Destination)
    os.remove(Destination + '.zip')
    
    
def OPPORTUNITY_Data_Loader(OPPORTUNITY_DATASET_FILE_LIST):
    df_OPPORTUNITY_DATASET_All = ''
    for file_name in OPPORTUNITY_DATASET_FILE_LIST:
        if file_name.endswith(".dat"):
            df_OPPORTUNITY_DATASET_current = pd.read_csv(OPPORTUNITY_DATASET_dataset_folder + '/' + file_name, sep = ' ', header = None)
            df_OPPORTUNITY_DATASET_current.columns = OPPORTUNITY_DATASET_column_names
            df_OPPORTUNITY_DATASET_current = df_OPPORTUNITY_DATASET_current.interpolate()
            df_OPPORTUNITY_DATASET_current.replace(np.nan, 0.0, inplace=True)
            df_OPPORTUNITY_DATASET_current['file_name'] = file_name
            if type(df_OPPORTUNITY_DATASET_All) != pd.core.frame.DataFrame:
                df_OPPORTUNITY_DATASET_All = df_OPPORTUNITY_DATASET_current
            else:
                df_OPPORTUNITY_DATASET_All = pd.concat([df_OPPORTUNITY_DATASET_All, df_OPPORTUNITY_DATASET_current])
    return df_OPPORTUNITY_DATASET_All

In [5]:
#import cfg file:

cfg_file_path = 'cfg.json'
cfg = import_cfg_file(cfg_file_path)

URL_OPPORTUNITY_DATASET = cfg['OPPORTUNITY_DATASET']['Download_URL']
Download_Destination_OPPORTUNITY_DATASET = cfg['OPPORTUNITY_DATASET']['Download_Destination']

URL_SKODA_DATASET = cfg['SKODA_DATASET']['Download_URL']
Download_Destination_SKODA_DATASET = cfg['SKODA_DATASET']['Download_Destination']

OPPORTUNITY_DATASET_dataset_folder = cfg['OPPORTUNITY_DATASET']['dataset_folder']

OPPORTUNITY_DATASET_Normalization_parameter = cfg['OPPORTUNITY_DATASET']['Normalization_parameter']

#download data: only if Dataset is not downloaded yet
"""
download_dataset(URL_OPPORTUNITY_DATASET, Download_Destination_OPPORTUNITY_DATASET)
download_dataset(URL_SKODA_DATASET, Download_Destination_SKODA_DATASET)
"""



100% [........................................................................] 84028489 / 84028489

''

In [5]:
OPPORTUNITY_DATASET_FILE_LIST = os.listdir(OPPORTUNITY_DATASET_dataset_folder)
OPPORTUNITY_DATASET_FILE_LIST

OPPORTUNITY_DATASET_FILE_LIST_Training = []
OPPORTUNITY_DATASET_FILE_LIST_Validation = []
OPPORTUNITY_DATASET_FILE_LIST_Testing = []


for file_name in OPPORTUNITY_DATASET_FILE_LIST:
    if file_name.endswith(".dat"):
        if 'S4' in file_name:
            continue
        if 'S1' in file_name:
            OPPORTUNITY_DATASET_FILE_LIST_Training.append(file_name)
        elif 'ADL3' in file_name:
            OPPORTUNITY_DATASET_FILE_LIST_Validation.append(file_name)
        elif ('ADL4' in file_name) or ('ADL5' in file_name):
            OPPORTUNITY_DATASET_FILE_LIST_Testing.append(file_name)
        else:
            OPPORTUNITY_DATASET_FILE_LIST_Training.append(file_name)

            
OPPORTUNITY_DATASET_FILE_LIST_Training
OPPORTUNITY_DATASET_FILE_LIST_Validation
OPPORTUNITY_DATASET_FILE_LIST_Testing

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'Dataset/OpportunityUCIDataset/OpportunityUCIDataset/dataset'

In [ ]:
#load column names
with open(OPPORTUNITY_DATASET_dataset_folder + '/column_names.txt') as txt_file:
    OPPORTUNITY_DATASET_column_name = txt_file.read().splitlines()

column_names = []
for column_name in OPPORTUNITY_DATASET_column_name:
    column_name = re.split('; |: ', column_name)
    if column_name[0] == 'Column':
        column_names.append(column_name[1])
    #print(column_name)
#OPPORTUNITY_DATASET_column_name
OPPORTUNITY_DATASET_column_names = column_names

In [ ]:
OPPORTUNITY_column_inclusive = ['InertialMeasurementUnit', 'Locomotion', 'ML_Both_Arms']
OPPORTUNITY_column_exclusive = ['Quaternion']

OPPORTUNITY_column_name_selected = OPPORTUNITY_DATASET_column_names[0: 37]
for column_name in OPPORTUNITY_DATASET_column_names:
    inclusive_index = 0
    exclusive_index = 1
    for keyword_inclusive in OPPORTUNITY_column_inclusive:
        if column_name.find(keyword_inclusive) != -1:
            inclusive_index = 1
            break
    for keyword_exclusive in OPPORTUNITY_column_exclusive:
        if column_name.find(keyword_exclusive) != -1:
            exclusive_index = 0
            break
    if inclusive_index * exclusive_index == 1:
        OPPORTUNITY_column_name_selected.append(column_name)

OPPORTUNITY_column_name_selected.append('file_name')
OPPORTUNITY_column_name_selected

In [ ]:
OPPORTUNITY_DATASET_column_names

In [ ]:
#load OPPORTUNITY_DATASET_Normalization_parameter
if os.path.isfile(OPPORTUNITY_DATASET_Normalization_parameter):
    df_OPPORTUNITY_DATASET_All_describe = pd.read_csv(OPPORTUNITY_DATASET_Normalization_parameter, index_col = 0)
else:
    df_OPPORTUNITY_DATASET_All = OPPORTUNITY_Data_Loader(OPPORTUNITY_DATASET_FILE_LIST)

    #df_OPPORTUNITY_DATASET_All.describe()
    df_OPPORTUNITY_DATASET_All_describe = df_OPPORTUNITY_DATASET_All.describe()
    #release memory
    df_OPPORTUNITY_DATASET_All = ''
    df_OPPORTUNITY_DATASET_All_describe.to_csv(OPPORTUNITY_DATASET_Normalization_parameter)
df_OPPORTUNITY_DATASET_All_describe

NORM_MAX_THRESHOLDS = [3000,   3000,   3000,   3000,   3000,   3000,   3000,   3000,   3000,
                       3000,   3000,   3000,   3000,   3000,   3000,   3000,   3000,   3000,
                       3000,   3000,   3000,   3000,   3000,   3000,   3000,   3000,   3000,
                       3000,   3000,   3000,   3000,   3000,   3000,   3000,   3000,   3000,
                       3000,   3000,   3000,   10000,  10000,  10000,  1500,   1500,   1500,
                       3000,   3000,   3000,   10000,  10000,  10000,  1500,   1500,   1500,
                       3000,   3000,   3000,   10000,  10000,  10000,  1500,   1500,   1500,
                       3000,   3000,   3000,   10000,  10000,  10000,  1500,   1500,   1500,
                       3000,   3000,   3000,   10000,  10000,  10000,  1500,   1500,   1500,
                       250,    25,     200,    5000,   5000,   5000,   5000,   5000,   5000,
                       10000,  10000,  10000,  10000,  10000,  10000,  250,    250,    25,
                       200,    5000,   5000,   5000,   5000,   5000,   5000,   10000,  10000,
                       10000,  10000,  10000,  10000,  250, ]

NORM_MIN_THRESHOLDS = [-3000,  -3000,  -3000,  -3000,  -3000,  -3000,  -3000,  -3000,  -3000,
                       -3000,  -3000,  -3000,  -3000,  -3000,  -3000,  -3000,  -3000,  -3000,
                       -3000,  -3000,  -3000,  -3000,  -3000,  -3000,  -3000,  -3000,  -3000,
                       -3000,  -3000,  -3000,  -3000,  -3000,  -3000,  -3000,  -3000,  -3000,
                       -3000,  -3000,  -3000,  -10000, -10000, -10000, -1000,  -1000,  -1000,
                       -3000,  -3000,  -3000,  -10000, -10000, -10000, -1000,  -1000,  -1000,
                       -3000,  -3000,  -3000,  -10000, -10000, -10000, -1000,  -1000,  -1000,
                       -3000,  -3000,  -3000,  -10000, -10000, -10000, -1000,  -1000,  -1000,
                       -3000,  -3000,  -3000,  -10000, -10000, -10000, -1000,  -1000,  -1000,
                       -250,   -100,   -200,   -5000,  -5000,  -5000,  -5000,  -5000,  -5000,
                       -10000, -10000, -10000, -10000, -10000, -10000, -250,   -250,   -100,
                       -200,   -5000,  -5000,  -5000,  -5000,  -5000,  -5000,  -10000, -10000,
                       -10000, -10000, -10000, -10000, -250, ]


df_OPPORTUNITY_DATASET_All_describe.loc[['min'], [col_name for col_name in OPPORTUNITY_column_name_selected[1: -3]]] = NORM_MIN_THRESHOLDS
df_OPPORTUNITY_DATASET_All_describe.loc[['max'], [col_name for col_name in OPPORTUNITY_column_name_selected[1: -3]]] = NORM_MAX_THRESHOLDS

df_OPPORTUNITY_DATASET_All_describe

In [ ]:
OPPORTUNITY_DATASET_FILE_LIST_Training
OPPORTUNITY_DATASET_FILE_LIST_Validation
OPPORTUNITY_DATASET_FILE_LIST_Testing


df_OPPORTUNITY_DATASET_Training = OPPORTUNITY_Data_Loader(OPPORTUNITY_DATASET_FILE_LIST_Training)
df_OPPORTUNITY_DATASET_Validation = OPPORTUNITY_Data_Loader(OPPORTUNITY_DATASET_FILE_LIST_Validation)
df_OPPORTUNITY_DATASET_Testing = OPPORTUNITY_Data_Loader(OPPORTUNITY_DATASET_FILE_LIST_Testing)


In [ ]:
rr = df_OPPORTUNITY_DATASET_Testing['33 Accelerometer LWR accY'].unique()
for m in rr:
    print(m)

In [ ]:
df_OPPORTUNITY_DATASET_Training.describe()

In [ ]:
df_OPPORTUNITY_DATASET_Testing.describe()

In [ ]:
df_OPPORTUNITY_DATASET_Validation.describe()

In [ ]:
df_OPPORTUNITY_DATASET_Training['file_name'].describe()

In [ ]:
df_OPPORTUNITY_DATASET_All_describe['2 Accelerometer RKN^ accX']['count']
df_OPPORTUNITY_DATASET_All_describe

In [ ]:
df_OPPORTUNITY_DATASET_Training['file_name']

In [ ]:


#len(OPPORTUNITY_column_name_selected)
#df_OPPORTUNITY_DATASET_WIP = df_OPPORTUNITY_DATASET[OPPORTUNITY_column_name_selected]

In [ ]:
OPPORTUNITY_column_name_selected

In [ ]:
def OPPORTUNITY_Normalization(df_OPPORTUNITY_DATASET, df_OPPORTUNITY_DS_All_describe, OPPORTUNITY_col_name_selected):
    for column_name in OPPORTUNITY_col_name_selected:
        if column_name in ['1 MILLISEC', '244 Locomotion', '250 ML_Both_Arms', 'file_name']:
            pass
        else:
            df_OPPORTUNITY_DATASET[column_name] = (df_OPPORTUNITY_DATASET[column_name] - df_OPPORTUNITY_DS_All_describe[column_name]['min']) / (df_OPPORTUNITY_DS_All_describe[column_name]['max'] - df_OPPORTUNITY_DS_All_describe[column_name]['min'])
            df_OPPORTUNITY_DATASET[column_name].loc[df_OPPORTUNITY_DATASET[column_name] > 1] = 1
            df_OPPORTUNITY_DATASET[column_name].loc[df_OPPORTUNITY_DATASET[column_name] < 0] = 0
            
    df_OPPORTUNITY_DATASET = df_OPPORTUNITY_DATASET[OPPORTUNITY_col_name_selected]
    return df_OPPORTUNITY_DATASET
    
    
df_OPPORTUNITY_DATASET_Training_WIP = OPPORTUNITY_Normalization(df_OPPORTUNITY_DATASET_Training, df_OPPORTUNITY_DATASET_All_describe, OPPORTUNITY_column_name_selected)
df_OPPORTUNITY_DATASET_Testing_WIP = OPPORTUNITY_Normalization(df_OPPORTUNITY_DATASET_Testing, df_OPPORTUNITY_DATASET_All_describe, OPPORTUNITY_column_name_selected)
df_OPPORTUNITY_DATASET_Validation_WIP = OPPORTUNITY_Normalization(df_OPPORTUNITY_DATASET_Validation, df_OPPORTUNITY_DATASET_All_describe, OPPORTUNITY_column_name_selected)


Classifications = df_OPPORTUNITY_DATASET_Training_WIP['250 ML_Both_Arms'].unique()
dict_18_classification = {}
class_index = 0
for Classitication in Classifications:
    dict_18_classification[Classitication] = class_index
    class_index += 1

df_OPPORTUNITY_DATASET_Training_WIP['250 ML_Both_Arms'] = df_OPPORTUNITY_DATASET_Training_WIP['250 ML_Both_Arms'].apply(lambda  x: dict_18_classification[x])
df_OPPORTUNITY_DATASET_Testing_WIP['250 ML_Both_Arms'] = df_OPPORTUNITY_DATASET_Testing_WIP['250 ML_Both_Arms'].apply(lambda  x: dict_18_classification[x])
df_OPPORTUNITY_DATASET_Validation_WIP['250 ML_Both_Arms'] = df_OPPORTUNITY_DATASET_Validation_WIP['250 ML_Both_Arms'].apply(lambda  x: dict_18_classification[x])

df_OPPORTUNITY_DATASET_Validation_WIP

In [ ]:

SILDE_WINDOW = 24
SILDE_WINDOW_STEP = 12


def data_generator(df_OPPORTUNITY_DATASET, SILDE_WINDOW, SILDE_WINDOW_STEP, OPPORTUNITY_column_name_selected):
    file_no = 0
    for file_name in df_OPPORTUNITY_DATASET['file_name'].unique():
        file_no += 1
        print(file_no)
        df_OPPORTUNITY_DATASET_WIP = df_OPPORTUNITY_DATASET.loc[df_OPPORTUNITY_DATASET['file_name'] == file_name]
        df_OPPORTUNITY_DATASET_WIP = df_OPPORTUNITY_DATASET_WIP[OPPORTUNITY_column_name_selected[1: ]]
        row_no_start = 0
        
        
        while (row_no_start + SILDE_WINDOW - 1) <= df_OPPORTUNITY_DATASET_WIP['2 Accelerometer RKN^ accX'].count():
            row_no_end = row_no_start + SILDE_WINDOW
            
            df_input = df_OPPORTUNITY_DATASET_WIP[OPPORTUNITY_column_name_selected[1: -3]][row_no_start: row_no_end]
            current_input = torch.tensor([df_input.values])
            
            df_output_Locomotion = df_OPPORTUNITY_DATASET_WIP['244 Locomotion'][row_no_end - 1]
            
            current_output_Locomotion = torch.tensor([df_output_Locomotion])
            #print(current_output_Locomotion)
            df_output_ML_Both_Arms = df_OPPORTUNITY_DATASET_WIP['250 ML_Both_Arms'][row_no_end - 1]
            current_output_ML_Both_Arms = torch.tensor([df_output_ML_Both_Arms])
            
            
            if row_no_start == 0 and file_no == 1:
                input_tensor = current_input
                output_Locomotion_tensor = current_output_Locomotion
                output_ML_Both_Arms_tensor = current_output_ML_Both_Arms
            else:
                input_tensor = torch.cat((input_tensor, current_input), 0)
                output_Locomotion_tensor = torch.cat((output_Locomotion_tensor, current_output_Locomotion), 0)
                output_ML_Both_Arms_tensor = torch.cat((output_ML_Both_Arms_tensor, current_output_ML_Both_Arms), 0)
                
            row_no_start += SILDE_WINDOW_STEP
            #print(input_tensor.size())
            #print(input_tensor)
    return input_tensor, output_Locomotion_tensor, output_ML_Both_Arms_tensor


"""
Training_x, Training_y_1, Training_y_2 = data_generator(df_OPPORTUNITY_DATASET_Training_WIP, SILDE_WINDOW, SILDE_WINDOW_STEP, OPPORTUNITY_column_name_selected)
Training_x = torch.reshape(Training_x, (Training_x.size()[0], 1, Training_x.size()[1], Training_x.size()[2])).float()
"""
"""
Testing_x, Testing_y_1, Testing_y_2 = data_generator(df_OPPORTUNITY_DATASET_Testing_WIP, SILDE_WINDOW, SILDE_WINDOW_STEP, OPPORTUNITY_column_name_selected)
Testing_x = torch.reshape(Testing_x, (Testing_x.size()[0], 1, Testing_x.size()[1], Testing_x.size()[2])).float()

Validation_x, Validation_y_1, Validation_y_2 = data_generator(df_OPPORTUNITY_DATASET_Validation_WIP, SILDE_WINDOW, SILDE_WINDOW_STEP, OPPORTUNITY_column_name_selected)
Validation_x = torch.reshape(Validation_x, (Validation_x.size()[0], 1, Validation_x.size()[1], Validation_x.size()[2])).float()
"""
#def batch_generator():


In [ ]:
#df_OPPORTUNITY_DATASET_Validation_WIP[OPPORTUNITY_column_name_selected[1: -3]][1: 2]

In [ ]:
#df_OPPORTUNITY_DATASET_WIP['22 Accelerometer RKN_ accZ'].unique()

In [116]:
def tensor_to_csv(tensor_name, file_name):
    x_np = tensor_name.cpu().detach().numpy()
    x_df = pd.DataFrame(x_np)
    x_df.to_csv('export/' + file_name + '.csv')

In [7]:
#import lasagne
#import theano
import time

import numpy as np
import _pickle as cp
#import theano.tensor as T

def load_dataset(filename):

    f = open(filename, 'rb')
    data = cp.load(f)
    f.close()

    X_train, y_train = data[0]
    X_test, y_test = data[1]

    print(" ..from file {}".format(filename))
    print(" ..reading instances: train {0}, test {1}".format(X_train.shape, X_test.shape))

    X_train = X_train.astype(np.float32)
    X_test = X_test.astype(np.float32)

    # The targets are casted to int8 for GPU compatibility.
    y_train = y_train.astype(np.uint8)
    y_test = y_test.astype(np.uint8)

    return X_train, y_train, X_test, y_test


filename = 'C:/Users/HX/Magic/oppChallenge_gestures.data'
X_train, y_train, X_test, y_test = load_dataset(filename)


 ..from file C:/Users/HX/Magic/oppChallenge_gestures.data
 ..reading instances: train (557963, 113), test (118750, 113)


In [57]:
pd.DataFrame(X_train[0:5000]).to_csv('sdf.csv')

In [ ]:
SILDE_WINDOW = 24
SILDE_WINDOW_STEP = 12


def data_generator(ARRAY_OPPORTUNITY_DATASET_x, ARRAY_OPPORTUNITY_DATASET_y, SILDE_WINDOW, SILDE_WINDOW_STEP):
    row_no_start = 0
    count_index = 0
    while (row_no_start + SILDE_WINDOW - 1) <= len(ARRAY_OPPORTUNITY_DATASET_x):
        count_index += 1
        if count_index % 500 == 0:
            print(count_index)
        row_no_end = row_no_start + SILDE_WINDOW
        df_input = ARRAY_OPPORTUNITY_DATASET_x[row_no_start: row_no_end]
        current_input = torch.tensor([df_input])
        df_output_Locomotion = ARRAY_OPPORTUNITY_DATASET_y[row_no_end - 1]
        df_output_ML_Both_Arms = torch.tensor([df_output_Locomotion])
        current_output_ML_Both_Arms = torch.tensor([df_output_ML_Both_Arms])


        if row_no_start == 0:
            input_tensor = current_input
            output_ML_Both_Arms_tensor = current_output_ML_Both_Arms
        else:
            input_tensor = torch.cat((input_tensor, current_input), 0)
            output_ML_Both_Arms_tensor = torch.cat((output_ML_Both_Arms_tensor, current_output_ML_Both_Arms), 0)
        row_no_start += SILDE_WINDOW_STEP
    return input_tensor, output_ML_Both_Arms_tensor


#X_train_tensor_1, y_train_tensor_1 = data_generator(X_train[0: 40000], y_train[0: 40000], SILDE_WINDOW, SILDE_WINDOW_STEP)


In [2]:
SILDE_WINDOW = 24
SILDE_WINDOW_STEP = 12

def data_generator(ARRAY_OPPORTUNITY_DATASET_x, ARRAY_OPPORTUNITY_DATASET_y, SILDE_WINDOW, SILDE_WINDOW_STEP):
    ARRAY_OPPORTUNITY_DATASET_x = torch.tensor(ARRAY_OPPORTUNITY_DATASET_x).float()
    ARRAY_OPPORTUNITY_DATASET_y = torch.tensor(ARRAY_OPPORTUNITY_DATASET_y)
    print(type(ARRAY_OPPORTUNITY_DATASET_x))
    array_len = len(ARRAY_OPPORTUNITY_DATASET_x)
    shape_0 = int(array_len / SILDE_WINDOW)
    shape_1 = SILDE_WINDOW
    shape_2 = len(ARRAY_OPPORTUNITY_DATASET_x[0])
    input_tensor = torch.reshape(ARRAY_OPPORTUNITY_DATASET_x[0: shape_0 * SILDE_WINDOW], (shape_0, shape_1, shape_2))
    print(type(input_tensor))
    output_tensor = torch.reshape(ARRAY_OPPORTUNITY_DATASET_y[0: shape_0 * SILDE_WINDOW], (shape_0, shape_1))
    
    output_tensor = output_tensor[:,-1]

    ARRAY_OPPORTUNITY_DATASET_x = ARRAY_OPPORTUNITY_DATASET_x[SILDE_WINDOW_STEP: ]
    array_len = len(ARRAY_OPPORTUNITY_DATASET_x)
    shape_0 = int(array_len / SILDE_WINDOW)
    shape_1 = SILDE_WINDOW
    shape_2 = len(ARRAY_OPPORTUNITY_DATASET_x[0])
    input_tensor = torch.cat((input_tensor, torch.reshape(ARRAY_OPPORTUNITY_DATASET_x[0: shape_0 * SILDE_WINDOW], (shape_0, shape_1, shape_2))), 0)
    
    ARRAY_OPPORTUNITY_DATASET_y = ARRAY_OPPORTUNITY_DATASET_y[SILDE_WINDOW_STEP: ]
    
    output_tensor_2 = torch.reshape(ARRAY_OPPORTUNITY_DATASET_y[0: shape_0 * SILDE_WINDOW], (shape_0, shape_1))
    output_tensor = torch.cat((output_tensor, output_tensor_2[:,-1]), 0)
    
    return input_tensor, output_tensor
    
    
X_train_tensor, y_train_tensor = data_generator(X_train, y_train, SILDE_WINDOW, SILDE_WINDOW_STEP)

pd.DataFrame(y_train_tensor.numpy()).to_csv('export/' + 'y_target' + '.csv')

df_target = pd.DataFrame(y_train_tensor.numpy())
df_target.columns = ['classification']
class_count = df_target['classification'].value_counts()
class_count = class_count.sort_index(0)
class_count = class_count.tolist()
class_count = 1 / (np.array(class_count) / (sum(class_count) / len(class_count)))
class_count





print(class_count)
#df_target['abc'].nunique()
#df_target
#print(X_train_tensor[0])
#input_tensor.size()
#output_tensor.size()
#print(type(class_count))

#Class_index = list(class_count.index.values)



pd.DataFrame(X_train_tensor[0]).to_csv('0.csv')

pd.DataFrame(y_train).to_csv('y.csv')

Validation_x = X_train_tensor
Validation_x = torch.reshape(Validation_x, (Validation_x.size()[0], 1, Validation_x.size()[1], Validation_x.size()[2])).float()

Validation_y_2 = y_train_tensor


pd.DataFrame(Validation_x[0, 0, :, :]).to_csv('0v.csv')

NameError: name 'X_train' is not defined

In [3]:
cfg_update = 1
if cfg_update == 1:
    cfg = {}
    cfg['OPPORTUNITY_DATASET'] = {}
    cfg['OPPORTUNITY_DATASET']['Download_URL'] = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00226/OpportunityUCIDataset.zip'
    cfg['OPPORTUNITY_DATASET']['Download_Destination'] = 'Dataset/OpportunityUCIDataset'
    cfg['OPPORTUNITY_DATASET']['dataset_folder'] = 'Dataset/OpportunityUCIDataset/OpportunityUCIDataset/dataset'
    cfg['OPPORTUNITY_DATASET']['Normalization_parameter'] = 'Dataset/OPPORTUNITY_norm_parameter.csv'
    
    cfg['SKODA_DATASET'] = {}
    cfg['SKODA_DATASET']['Download_URL'] = 'http://har-dataset.org/lib/exe/fetch.php?media=wiki:dataset:skodaminicp:skodaminicp_2015_08.zip'
    cfg['SKODA_DATASET']['Download_Destination'] = 'Dataset/SKODA_DATASET'
    cfg['SKODA_DATASET']['dataset_path'] = 'Dataset/SKODA_DATASET/SkodaMiniCP_2015_08/right_classall_clean.mat'
    
    
    
    
    
    
    #data = json.dumps(data, indent=4)
    #print(data)
    with open('cfg.json', 'w') as outfile:
        json.dump(cfg, outfile)



SKODA_DATASET_dataset_path = cfg['SKODA_DATASET']['dataset_path']

import scipy.io
right_classall_clean_mat = scipy.io.loadmat(SKODA_DATASET_dataset_path)
right_classall_clean_array = right_classall_clean_mat['right_classall_clean']

SKODA_DATASET_dataset_size = right_classall_clean_array.shape[0]


SKODA_Selected_row = [int(k * (98 / 30)) for k in range(0, int(SKODA_DATASET_dataset_size / (98 / 30)) + 1)]
#print(SKODA_Selected_row)

no_of_sensor = 10
SKODA_Selected_column_x = [2 + s * 7 for s in range(no_of_sensor)] + [3 + s * 7 for s in range(no_of_sensor)] + [4 + s * 7 for s in range(no_of_sensor)]
SKODA_Selected_column_x.sort()
SKODA_Selected_column_y = 0

right_classall_clean_array = right_classall_clean_array[SKODA_Selected_row,:].astype(np.float)

right_classall_clean_array_x = right_classall_clean_array[:, SKODA_Selected_column_x]
right_classall_clean_array_y = right_classall_clean_array[:, SKODA_Selected_column_y]


print(right_classall_clean_array_x.shape)
print(SKODA_Selected_column_x)
right_classall_clean_array
SKODA_Selected_column_norm = []
for column in range(len(right_classall_clean_array_x[0])):
    max_value = max(right_classall_clean_array_x[:, column])
    min_value = min(right_classall_clean_array_x[:, column])
    SKODA_Selected_column_norm.append([max_value, min_value, (max_value - min_value)])
    #right_classall_clean_array_x[:, column] = (right_classall_clean_array_x[:, column] - min_value) / (max_value - min_value)
    
len(SKODA_Selected_column_norm)

right_classall_clean_array_x.shape


SKODA_original_class = np.unique(right_classall_clean_array_y)
dict_SKODA_class = {}
for i in range(len(SKODA_original_class)):
    dict_SKODA_class[SKODA_original_class[i]] = i
for i in range(len(right_classall_clean_array_y)):
    right_classall_clean_array_y[i] = dict_SKODA_class[right_classall_clean_array_y[i]]
    



X_train_tensor, y_train_tensor = data_generator(right_classall_clean_array_x, right_classall_clean_array_y, SILDE_WINDOW, SILDE_WINDOW_STEP)

(216094, 30)
[2, 3, 4, 9, 10, 11, 16, 17, 18, 23, 24, 25, 30, 31, 32, 37, 38, 39, 44, 45, 46, 51, 52, 53, 58, 59, 60, 65, 66, 67]
<class 'torch.Tensor'>
<class 'torch.Tensor'>


In [137]:
y_train_tensor.unique()
X_train_tensor

tensor([[[ -460.,  -788.,   158.,  ...,   420.,  -778.,  -544.],
         [ -460.,  -788.,   158.,  ...,   430.,  -778.,  -544.],
         [ -460.,  -788.,   168.,  ...,   430.,  -788.,  -534.],
         ...,
         [ -550.,  -769.,   118.,  ...,   510.,  -788.,  -495.],
         [ -540.,  -769.,    99.,  ...,   490.,  -798.,  -514.],
         [ -550.,  -807.,    79.,  ...,   490.,  -807.,  -504.]],

        [[ -570.,  -826.,    69.,  ...,   520.,  -826.,  -495.],
         [ -560.,  -846.,    39.,  ...,   560.,  -875.,  -455.],
         [ -530.,  -846.,    59.,  ...,   490.,  -875.,  -465.],
         ...,
         [ -430.,  -750.,   178.,  ...,   360.,  -759.,  -554.],
         [ -450.,  -769.,   158.,  ...,   360.,  -778.,  -554.],
         [ -420.,  -807.,   168.,  ...,   310.,  -788.,  -564.]],

        [[ -390.,  -884.,   148.,  ...,   320.,  -846.,  -554.],
         [ -410.,  -942.,    79.,  ...,   400.,  -932.,  -514.],
         [ -350.,  -903.,    49.,  ...,   350.,  -923.,  -

In [97]:

y_train_tensor.unique()
#torch.save({'X_train_tensor': X_train_tensor, 'y_train_tensor': y_train_tensor}, 'C:/Users/HX/Magic/training_data.pt')

KeyError: tensor(32., dtype=torch.float64)

In [106]:

np.unique(right_classall_clean_array_y)
#dict_SKODA_class

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.])

In [ ]:
"""
data_loading = torch.load("D:/Installation/yolov4.pt")
X_train_tensor = data_loading['X_train_tensor']
y_train_tensor = data_loading['y_train_tensor']
data_loading = ''
"""

In [ ]:
#X_train_tensor.size()

pending items:
    - batch size = 100
    - sensor data were pre-processed to fill in missing values using linear interpolation
    - model accuracy & loss function save into array
    - Dashboard - to show performance
    
    - per channel normalization to interval [0,1]
    https://stackoverflow.com/questions/26414913/normalize-columns-of-pandas-data-frame

In [4]:
"""
input = np.array([random() for i in range(24 * 113 * 1 * 6)]).reshape(6, 1, 24, 113)

output = torch.Tensor([[1.0, 0.0, 0.0, 0.0, 0.0],
                     [0.0, 1.0, 0.0, 0.0, 0.0],
                     [0.0, 0.0, 0.0, 1.0, 0.0],
                     [0.0, 0.0, 1.0, 0.0, 0.0],
                     [0.0, 0.0, 1.0, 0.0, 0.0],
                     [0.0, 0.0, 0.0, 0.0, 1.0]])


output = torch.Tensor([0, 1, 3, 2, 2, 4])

final_input = torch.Tensor(input)
final_output = torch.Tensor(output)
"""

class CNN_LSTM_HAR_MODEL(nn.Module):
    def __init__(self):
        super().__init__()
        self.cnn2d_1 = nn.Conv2d(in_channels = 1, out_channels = 64, kernel_size = (5, 1))
        #self.conv_bn = nn.BatchNorm2d(64)
        #self.relu = nn.ReLU()
        self.cnn2d_2 = nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size = (5, 1))
        self.cnn2d_3 = nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size = (5, 1))
        self.cnn2d_4 = nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size = (5, 1))
        
        self.batch_size = 24 - 4 * (5 - 1)
        self.hidden_dim = 128
        self.input_size = 7232
        self.input_size = 1920
        self.n_layers = 2
        
        self.class_no = 11
        
        #self.lstm1 = nn.LSTM(batch_first = True, input_size  = 7232, hidden_size = self.hidden_dim, num_layers = self.n_layers)
        self.lstm1 = nn.LSTM(batch_first = True, input_size  = self.input_size, hidden_size = self.hidden_dim, dropout = 0.5)
        self.lstm2 = nn.LSTM(batch_first = True, input_size  = 128, hidden_size = self.hidden_dim, dropout = 0.5)
        
        #self.lstm1 = nn.LSTM(batch_first = True, input_size  = 7232, hidden_size = self.hidden_dim)
        #self.lstm2 = nn.LSTM(batch_first = True, input_size  = 128, hidden_size = self.hidden_dim)
        
        #self.conv_bn2 = nn.BatchNorm1d(8)
        #, dropout = 0.5
        #self.input_size = 128
        #self.n_layers = 1
        #self.lstm2 = nn.LSTM(input_size  = self.input_size, hidden_size = self.hidden_dim, num_layers = self.n_layers, dropout = 0.5)
        
        #self.fc1 = nn.Linear(128, 18)
        self.fc1 = nn.Linear(128, self.class_no)
        
    def forward(self, x):
        #print(x)
        x = self.cnn2d_1(x)
        #x = self.relu(x)
        #x = self.conv_bn(x)
        #tensor_to_csv(x[0, -1, :, :], 'layer_1_data_0')
        #tensor_to_csv(x[5, -1, :, :], 'layer_1_data_100')
        
        x = self.cnn2d_2(x)
        #x = self.relu(x)
        #x = self.conv_bn(x)
        #tensor_to_csv(x[0, -1, :, :], 'layer_2_data_0')
        #tensor_to_csv(x[5, -1, :, :], 'layer_2_data_100')

        x = self.cnn2d_3(x)
        #x = self.relu(x)
        #x = self.conv_bn(x)
        #tensor_to_csv(x[0, -1, :, :], 'layer_3_data_0')
        #tensor_to_csv(x[5, -1, :, :], 'layer_3_data_100')

        x = self.cnn2d_4(x)
        #x = self.relu(x)
        #x = self.conv_bn(x)
        #tensor_to_csv(x[0, -1, :, :], 'layer_4_data_0')
        #tensor_to_csv(x[5, -1, :, :], 'layer_4_data_100')

        #print(x.size())
        x = torch.transpose(x, 1, 2)
        #print(x.size())
        x = torch.reshape(x, (x.size()[0], x.size()[1], -1))

        #hidden = self.init_hidden()
        #hidden = (torch.randn(2, x.size()[0], self.hidden_dim).cuda(), torch.randn(2, x.size()[0], self.hidden_dim).cuda())
        #x, hidden = self.lstm1(x, hidden)
        x, hidden = self.lstm1(x)
        x, hidden = self.lstm2(x)
        #print(x)
        #x = self.conv_bn2(x)
        #hidden = self.conv_bn2(hidden)
        #tensor_to_csv(x[0, -1, :,], 'layer_lstm1_data_0')
        #tensor_to_csv(x[5, -1, :], 'layer_lstm1_data_100')
        #print(x)
        #hidden2 = (torch.zeros(self.n_layers, 8, self.hidden_dim).requires_grad_(), torch.zeros(self.n_layers, 8, self.hidden_dim).requires_grad_())
        ###############################
        #x, hidden = self.lstm2(x, hidden)
        #print(x)
        #x = self.conv_bn2(x)
        
        #tensor_to_csv(x[0, -1, :,], 'layer_lstm2_data_0')
        #tensor_to_csv(x[15, -1, :], 'layer_lstm2_data_100')
        #print(x.size())
        x = torch.reshape(x, (-1, 128))
        #print(x.size())
        
        x = self.fc1(x)
        x = F.softmax(x, dim = 1)
        #x = F.softmax(x)
        x = torch.reshape(x, (int(x.size()[0] / 8), 8, self.class_no))
        x = x[:, -1, :]
        #print(x.size())
        tensor_to_csv(x, 'layer_fc1_data')
        
        
        tensor_to_csv(x, 'output_data')
        
        
        #print(x)
        return x
    
    def init_hidden(self):
        weight = next(self.parameters()).data

        hidden = (weight.new(self.n_layers, self.batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, self.batch_size, self.hidden_dim).zero_())
        return hidden

    
    
    
    
    




In [5]:

class CNN_LSTM_HAR_MODEL(nn.Module):
    def __init__(self):
        super().__init__()
        self.cnn2d_1 = nn.Conv2d(in_channels = 1, out_channels = 64, kernel_size = (5, 1))
        self.conv_bn = nn.BatchNorm2d(64)
        #self.relu = nn.ReLU()
        self.cnn2d_2 = nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size = (5, 1))
        self.cnn2d_3 = nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size = (5, 1))
        self.cnn2d_4 = nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size = (5, 1))
        
        self.batch_size = 24 - 4 * (5 - 1)
        self.hidden_dim = 128
        self.input_size = 7232
        self.input_size = 1920
        self.n_layers = 2
        
        self.class_no = 11
        
        #self.lstm1 = nn.LSTM(batch_first = True, input_size  = 7232, hidden_size = self.hidden_dim, num_layers = self.n_layers)
        self.lstm1 = nn.LSTM(batch_first = True, input_size  = self.input_size, hidden_size = self.hidden_dim, dropout = 0.5)
        self.lstm2 = nn.LSTM(batch_first = True, input_size  = 128, hidden_size = self.hidden_dim, dropout = 0.5)
        self.lstm_bn = nn.BatchNorm1d(8)
        #self.lstm1 = nn.LSTM(batch_first = True, input_size  = 7232, hidden_size = self.hidden_dim)
        #self.lstm2 = nn.LSTM(batch_first = True, input_size  = 128, hidden_size = self.hidden_dim)
        
        #self.conv_bn2 = nn.BatchNorm1d(8)
        #, dropout = 0.5
        #self.input_size = 128
        #self.n_layers = 1
        #self.lstm2 = nn.LSTM(input_size  = self.input_size, hidden_size = self.hidden_dim, num_layers = self.n_layers, dropout = 0.5)
        
        #self.fc1 = nn.Linear(128, 18)
        self.fc1 = nn.Linear(128, self.class_no)
        
    def forward(self, x):
        #print(x)
        x = self.cnn2d_1(x)
        #x = self.relu(x)
        x = self.conv_bn(x)
        #tensor_to_csv(x[0, -1, :, :], 'layer_1_data_0')
        #tensor_to_csv(x[5, -1, :, :], 'layer_1_data_100')
        
        x = self.cnn2d_2(x)
        #x = self.relu(x)
        x = self.conv_bn(x)
        #tensor_to_csv(x[0, -1, :, :], 'layer_2_data_0')
        #tensor_to_csv(x[5, -1, :, :], 'layer_2_data_100')

        x = self.cnn2d_3(x)
        #x = self.relu(x)
        x = self.conv_bn(x)
        #tensor_to_csv(x[0, -1, :, :], 'layer_3_data_0')
        #tensor_to_csv(x[5, -1, :, :], 'layer_3_data_100')

        x = self.cnn2d_4(x)
        #x = self.relu(x)
        x = self.conv_bn(x)
        #tensor_to_csv(x[0, -1, :, :], 'layer_4_data_0')
        #tensor_to_csv(x[5, -1, :, :], 'layer_4_data_100')

        #print(x.size())
        x = torch.transpose(x, 1, 2)
        #print(x.size())
        x = torch.reshape(x, (x.size()[0], x.size()[1], -1))

        #hidden = self.init_hidden()
        #hidden = (torch.randn(2, x.size()[0], self.hidden_dim).cuda(), torch.randn(2, x.size()[0], self.hidden_dim).cuda())
        #x, hidden = self.lstm1(x, hidden)
        x, hidden = self.lstm1(x)
        
        x = self.lstm_bn(x)
        
        x, hidden = self.lstm2(x)
        
        x = self.lstm_bn(x)
        
        #print(x)
        #x = self.conv_bn2(x)
        #hidden = self.conv_bn2(hidden)
        #tensor_to_csv(x[0, -1, :,], 'layer_lstm1_data_0')
        #tensor_to_csv(x[5, -1, :], 'layer_lstm1_data_100')
        #print(x)
        #hidden2 = (torch.zeros(self.n_layers, 8, self.hidden_dim).requires_grad_(), torch.zeros(self.n_layers, 8, self.hidden_dim).requires_grad_())
        ###############################
        #x, hidden = self.lstm2(x, hidden)
        #print(x)
        #x = self.conv_bn2(x)
        
        #tensor_to_csv(x[0, -1, :,], 'layer_lstm2_data_0')
        #tensor_to_csv(x[15, -1, :], 'layer_lstm2_data_100')
        #print(x.size())
        x = torch.reshape(x, (-1, 128))
        #print(x.size())
        
        x = self.fc1(x)
        x = F.softmax(x, dim = 1)
        #x = F.softmax(x)
        x = torch.reshape(x, (int(x.size()[0] / 8), 8, self.class_no))
        #x = x[:, -1, :]
        #print(x.size())
        #tensor_to_csv(x, 'layer_fc1_data')
        
        
        #tensor_to_csv(x, 'output_data')
        
        
        #print(x)
        return x[:, -1, :]



In [ ]:
#list(Validation_y_2.size())[0]

In [13]:

#y_true = [0, 1, 2, 0, 1, 2]
#y_pred = [0, 2, 1, 0, 0, 1]


BATCH_SIZE = 1200


CNN_LSTM_HAR_MODEL_WIP = CNN_LSTM_HAR_MODEL().cuda()
learning_rate = 0.001
#learning_rate = 0.02
#loss_functioin = nn.CrossEntropyLoss(weight = (torch.tensor([0.2/0.8/17, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0])/ (0.2/0.8/17 + 17)).cuda())
#loss_functioin = nn.CrossEntropyLoss(weight = torch.tensor(class_count).float().cuda())
#loss_functioin = nn.CrossEntropyLoss()

SKODA_weight = [0.386610555,
                1.024348618,
                0.874884602,
                0.909393939,
                1.26013017,
                2.056418456,
                2.188381138,
                1.18103109,
                1.079755337,
                0.92690209,
                1.650109971,]

loss_functioin = nn.CrossEntropyLoss(weight = torch.tensor(SKODA_weight).float().cuda())





#optimizer = optim.SGD(CNN_LSTM_HAR_MODEL_WIP.parameters(), lr = learning_rate)
#https://pytorch.org/docs/stable/optim.html
optimizer = torch.optim.RMSprop(CNN_LSTM_HAR_MODEL_WIP.parameters(), lr = learning_rate, weight_decay = 0.9)
CNN_LSTM_HAR_MODEL_WIP.load_state_dict(torch.load("model.pt")['state_dict'])
optimizer.load_state_dict(torch.load("model.pt")['optimizer'])
CNN_LSTM_HAR_MODEL_WIP.train()


Training_set = 'Validation'
Validation_x = X_train_tensor
Validation_x = torch.reshape(Validation_x, (Validation_x.size()[0], 1, Validation_x.size()[1], Validation_x.size()[2])).float()

Validation_y_2 = y_train_tensor


#output = torch.Tensor([0, 1, 3, 2, 2, 4])

#final_input = torch.Tensor(input)
#final_output = torch.Tensor(output)





epoch_size = 300
steps_for_printing_out_loss = 1

data_size = list(Validation_y_2.size())[0]
torch.cuda.empty_cache()
for i in range(1, epoch_size + 1):
    optimizer.zero_grad()
    torch.cuda.empty_cache()
    no_of_right = 0
    epoch_loss = 0
    for Batch_no in range(int(data_size / BATCH_SIZE) + 1):
        """
        if Batch_no % 80 == 1:
            print("Batch {}/{}".format(str(Batch_no), int(data_size / BATCH_SIZE)))
        """
        batch_start = Batch_no * BATCH_SIZE
        batch_end = min(data_size, (Batch_no + 1) * BATCH_SIZE)

        if Training_set == 'Validation':
            input = Validation_x[batch_start: batch_end].cuda()
            target = Validation_y_2[batch_start: batch_end].cuda()
        elif Training_set == 'Testing':
            input = Testing_x[batch_start: batch_end].cuda()
            target = Testing_y_2[batch_start: batch_end].cuda()
        elif Training_set == 'Training':
            input = Training_x[batch_start: batch_end].cuda()
            target = Training_y_2[batch_start: batch_end].cuda()
        
        output = CNN_LSTM_HAR_MODEL_WIP(input).cuda()
        loss = loss_functioin(output, target.long()).cuda()
        epoch_loss += loss
        loss.backward()
        if i == 1 or i % (steps_for_printing_out_loss) == 0:
            #print('Loss (epoch: ' + str(i) + '): ' + str(loss.cpu().detach().numpy()))
            
            if Batch_no == 0:
                All_output = output.argmax(1)
            else:
                All_output = torch.cat((All_output, output.argmax(1)), 0)
            
            
            for k in range(len(target)):
                if output.argmax(1)[k] == target[k]:
                    #print('--------')
                    #print(output[k])
                    #print(target[k])
                    no_of_right += 1
    if i == 1 or i % (steps_for_printing_out_loss) == 0:
        F_score = f1_score(y_train_tensor.numpy(), All_output.cpu().detach().numpy(), average='weighted')
        
        pd.DataFrame(y_train_tensor.numpy()).to_csv('export/' + 'y_target' + '.csv')
        pd.DataFrame(All_output.cpu().detach().numpy()).to_csv('export/' + 'y_predict' + '.csv')
        #print(CNN_LSTM_HAR_MODEL_WIP.cnn2d_3.weight.grad)
        print("Epoch {}/{}, Loss: {:.6f}, F_score: {:.6f}, Accuracy: {:.6f}".format(i, epoch_size, epoch_loss.cpu().detach().numpy(), F_score, no_of_right / data_size))
    optimizer.step()
torch.save({'state_dict': CNN_LSTM_HAR_MODEL_WIP.state_dict(),'optimizer': optimizer.state_dict()}, 'model.pt')

Epoch 1/300, Loss: 28.921093, F_score: 0.691294, Accuracy: 0.698045
Epoch 2/300, Loss: 28.888231, F_score: 0.715622, Accuracy: 0.720371
Epoch 3/300, Loss: 28.965012, F_score: 0.686665, Accuracy: 0.693324
Epoch 4/300, Loss: 28.932266, F_score: 0.711846, Accuracy: 0.716872
Epoch 5/300, Loss: 29.002560, F_score: 0.688857, Accuracy: 0.693769
Epoch 6/300, Loss: 28.965506, F_score: 0.711012, Accuracy: 0.716039
Epoch 7/300, Loss: 28.955309, F_score: 0.691981, Accuracy: 0.696157
Epoch 8/300, Loss: 28.936441, F_score: 0.706939, Accuracy: 0.713318
Epoch 9/300, Loss: 28.956451, F_score: 0.696665, Accuracy: 0.699211
Epoch 10/300, Loss: 28.909204, F_score: 0.716380, Accuracy: 0.721093
Epoch 11/300, Loss: 28.916250, F_score: 0.696169, Accuracy: 0.699656
Epoch 12/300, Loss: 28.917425, F_score: 0.724908, Accuracy: 0.727591
Epoch 13/300, Loss: 29.032864, F_score: 0.686800, Accuracy: 0.690603
Epoch 14/300, Loss: 28.960979, F_score: 0.724459, Accuracy: 0.726869
Epoch 15/300, Loss: 28.967543, F_score: 0.6

Epoch 120/300, Loss: 28.836523, F_score: 0.729411, Accuracy: 0.732311
Epoch 121/300, Loss: 28.861338, F_score: 0.702055, Accuracy: 0.705543
Epoch 122/300, Loss: 28.818707, F_score: 0.732452, Accuracy: 0.735310
Epoch 123/300, Loss: 28.853529, F_score: 0.702923, Accuracy: 0.705987
Epoch 124/300, Loss: 28.811640, F_score: 0.734641, Accuracy: 0.737254
Epoch 125/300, Loss: 28.835512, F_score: 0.703832, Accuracy: 0.707098
Epoch 126/300, Loss: 28.795160, F_score: 0.738271, Accuracy: 0.740087
Epoch 127/300, Loss: 28.821512, F_score: 0.703622, Accuracy: 0.707209
Epoch 128/300, Loss: 28.775209, F_score: 0.742209, Accuracy: 0.743585
Epoch 129/300, Loss: 28.811415, F_score: 0.703728, Accuracy: 0.707597
Epoch 130/300, Loss: 28.763941, F_score: 0.741196, Accuracy: 0.743030
Epoch 131/300, Loss: 28.837019, F_score: 0.702554, Accuracy: 0.706209
Epoch 132/300, Loss: 28.818804, F_score: 0.733396, Accuracy: 0.736810
Epoch 133/300, Loss: 28.957258, F_score: 0.699432, Accuracy: 0.704598
Epoch 134/300, Loss:

Epoch 238/300, Loss: 28.718853, F_score: 0.706097, Accuracy: 0.710097
Epoch 239/300, Loss: 28.650375, F_score: 0.750742, Accuracy: 0.751472
Epoch 240/300, Loss: 28.710169, F_score: 0.708105, Accuracy: 0.712040
Epoch 241/300, Loss: 28.652876, F_score: 0.750583, Accuracy: 0.751527
Epoch 242/300, Loss: 28.720888, F_score: 0.707099, Accuracy: 0.711152
Epoch 243/300, Loss: 28.697025, F_score: 0.747498, Accuracy: 0.748750
Epoch 244/300, Loss: 28.798187, F_score: 0.701713, Accuracy: 0.705432
Epoch 245/300, Loss: 28.745796, F_score: 0.741724, Accuracy: 0.743530
Epoch 246/300, Loss: 28.796701, F_score: 0.706434, Accuracy: 0.709208
Epoch 247/300, Loss: 28.734890, F_score: 0.741745, Accuracy: 0.743752
Epoch 248/300, Loss: 28.696682, F_score: 0.712869, Accuracy: 0.716039
Epoch 249/300, Loss: 28.608328, F_score: 0.757376, Accuracy: 0.757970
Epoch 250/300, Loss: 28.672997, F_score: 0.708261, Accuracy: 0.712762
Epoch 251/300, Loss: 28.649921, F_score: 0.760454, Accuracy: 0.758914
Epoch 252/300, Loss:

In [ ]:
def tensor_to_csv(tensor_name):
    x_np = tensor_name.numpy()
    x_df = pd.DataFrame(x_np)
    x_df.to_csv('tmp.csv')

Validation_x.size()
#output

tensor_to_csv(output.detach())


#Validation_x_to_csv = torch.reshape(Validation_x, (Validation_x.size()[0] * Validation_x.size()[1] * Validation_x.size()[2], Validation_x.size()[3])).float()
#tensor_to_csv(Validation_x_to_csv.detach())
#tensor_to_csv(Validation_x[1, 0,:, :])

#tensor_to_csv(Validation_y_2)

for row_no_end in range(0, 155):
    print(row_no_end)
    print(df_OPPORTUNITY_DATASET_Validation_WIP['250 ML_Both_Arms'][row_no_end])

In [ ]:
input.size()
target
CNN_LSTM_HAR_MODEL_WIP.parameters()
target
Validation_y_2.sum()
print(Validation_y_2)
for i in Validation_y_2.long():
    print(i)

In [ ]:
for i in CNN_LSTM_HAR_MODEL_WIP.parameters():
    print(i.size())
    print(i)

In [ ]:

torch.isnan(Validation_x)



x_np = Validation_x[0, 0,:, :].numpy()
x_df = pd.DataFrame(x_np)
x_df.to_csv('tmp.csv')





In [ ]:
>>> rnn = nn.LSTM(10, 20, 2)
>>> input = torch.randn(5, 3, 10)
>>> h0 = torch.randn(2, 3, 20)
>>> c0 = torch.randn(2, 3, 20)
>>> output, (hn, cn) = rnn(input, (h0, c0))

output.size()
#hn.size()

(216094, 30)
[2, 3, 4, 9, 10, 11, 16, 17, 18, 23, 24, 25, 30, 31, 32, 37, 38, 39, 44, 45, 46, 51, 52, 53, 58, 59, 60, 65, 66, 67]


NameError: name 'data_generator' is not defined

In [ ]:
kk = torch.tensor([[[k* 100 + j * 10 + i for i in range (10)] for j in range(10)] for k in range(10)])
kk
#mm = torch.reshape(kk, (kk.size()[0], 1, kk.size()[1], kk.size()[2])).float()
mm = torch.reshape(kk, (kk.size()[0], -1))


print(mm.size())

print(mm[0])


"""
mm[:, 0, :, :] == kk
mm[5, 0, 4, 3]
"""

In [ ]:
for para in CNN_LSTM_HAR_MODEL_WIP.parameters():
    print(para.size())

In [ ]:
df_OPPORTUNITY_DATASET_Training_WIP[OPPORTUNITY_column_name_selected][0:2000].to_csv('def.csv')

In [ ]:
0.2/0.8/17


In [49]:
for i in CNN_LSTM_HAR_MODEL_WIP.parameters():
    print(i.size())
    #print(i)
    
    

torch.Size([64, 1, 5, 1])
torch.Size([64])
torch.Size([64, 64, 5, 1])
torch.Size([64])
torch.Size([64, 64, 5, 1])
torch.Size([64])
torch.Size([64, 64, 5, 1])
torch.Size([64])
torch.Size([512, 7232])
torch.Size([512, 128])
torch.Size([512])
torch.Size([512])
torch.Size([512, 128])
torch.Size([512, 128])
torch.Size([512])
torch.Size([512])
torch.Size([18, 128])
torch.Size([18])
